## Preprocess.

### Load Labels.

In [4]:
import os
import numpy as np
import nibabel as nib
from mne import read_label
from pandas import read_csv
from pandas import read_table

## Specify directories and parameters.
fast_dir = '/space/will/3/users/EMBARC/EMBARC-FAST'
fsd = 'rest'
run = '001'

##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~## 
### Load cortical labels.
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~## 

parc_dir = '/space/lilli/1/users/DARPA-Recons/fscopy/label/yeo114_orig'
lh_labels = [read_label(os.path.join(parc_dir, l)) for l in sorted(os.listdir(parc_dir)) if 'LH' in l]
rh_labels = [read_label(os.path.join(parc_dir, l)) for l in sorted(os.listdir(parc_dir)) if 'RH' in l]

labels = np.append(lh_labels,rh_labels)

##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~## 
### Load subcortical labels.
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~## 

## Read Freesurfer Color Lookup Table. 
rois = ['Left-Accumbens-area', 'Left-Thalamus-Proper', 'Left-Caudate', 'Left-Putamen', 'Left-Hippocampus', 'Left-Amygdala', 
        'Right-Accumbens-area', 'Right-Thalamus-Proper', 'Right-Caudate', 'Right-Putamen', 'Right-Hippocampus', 'Right-Amygdala']

subcort_labels = dict.fromkeys([l for l in rois])

clut = read_table(os.path.join('/space/will/3/users/EMBARC/EMBARC-FAST/scripts/labels', 'FreeSurferColorLUT_truncated.txt'), 
                  sep=' *', skiprows=4, names=['No', 'Label','R','G','B','A'], engine='python')
clut = clut.loc[np.where(np.in1d(clut.Label,rois))]

## Load subcortical segmentation.
aseg_obj = nib.load('/space/lilli/1/users/DARPA-Recons/fsaverage/mri.2mm/aseg.mgz')
aseg_dat = aseg_obj.get_data()

for l in rois: subcort_labels[l] = np.where(aseg_dat == int(clut.No[clut.Label == l]))
print 'cortical lh: %d,' %len(lh_labels), 'rh: %d\n' %len(rh_labels), 'subcortical: %d' %len(subcort_labels)

cortical lh: 57, rh: 57
subcortical: 12


### Specify Subjects.

In [5]:
def normalize(arr):
    return [( float(i) - min(arr) )/( max(arr) - min(arr)) for i in arr] 

## Load questionnaires. 
csv = read_csv('/space/will/3/users/EMBARC/EMBARC-FAST/scripts/questionnaires/subtypes_qscores.csv')

## Restrict to questionnaires of interest.
csv.columns = ['Subject_ID', 'Diagnosis','MASQ_AA', 'SHAPS_Cont', 'AAQ']
csv = csv.set_index('Subject_ID', drop=True)

## Restrict to psychiatric subjects. 
csv = csv[csv['Diagnosis'] == 1]

## Remove subjects with corrupted data.
csv = csv[csv.index != 'CU0068CUMR1R1'] ## need to re-proproc

## Remove subjects with missing data.
csv = csv.dropna()
subjects = list(csv.index)

## Create questionnaire matrix. 
qmat = np.vstack([csv['MASQ_AA'], csv['SHAPS_Cont'], csv['AAQ']])

## Normalize questionnaires.
for i in np.arange(qmat.shape[0]):
    qmat[i] = normalize(qmat[i])
    
## Add intercept column. 
intercept = np.array([1]*len(subjects))
qmat = np.vstack([intercept,qmat])

## Save out list of subjects.
with open('/space/will/3/users/EMBARC/EMBARC-FAST/scripts/subtypes_gt/gt_subjects', 'w') as f:
    for s in subjects: 
        f.write(s+'\n')

### Load Data. Average by Labels.

In [16]:
import scipy.io as sio
from pandas import DataFrame
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from mne.filter import band_pass_filter

## Define functions and directories.
def zscore(arr): return (arr - arr.mean()) / arr.std()

## Specify directories. 
gt_dir = '/space/will/3/users/EMBARC/EMBARC-FAST/scripts/subtypes_gt'

## Define matrices.
n_preds, n_subs = qmat.shape
n_regions = len(labels) + len(subcort_labels)
subjects = [line.strip() for line in open("/space/will/3/users/EMBARC/EMBARC-FAST/scripts/subtypes_gt/gt_subjects", 'r')]

# ## Subjects with manual coregistration.
# subjects = ['CU0092CUMR1R1', 'CU0087CUMR1R1', 'CU0095CUMR1R1', 'CU0104CUMR1R1', 'CU0131CUMR1R1', 'CU0069CUMR1R1', 
#             'CU0025CUMR1R1', 'CU0094CUMR1R1', 'CU0067CUMR1R1', 'CU0106CUMR1R1', 'CU0009CUMR1R1', 'CU0105CUMR1R1', 
#             'CU0070CUMR1R1', 'CU0129CUMR1R1', 'CU0113CUMR1R1', 'CU0125CUMR1R1', 'CU0133CUMR1R1']

for i, subject in enumerate(subjects):
    
    print i, subject

    ## Load cortical/subcortical data.
    lh_dat = (nib.load(os.path.join(fast_dir, subject, fsd, run, 'fmcpr.sm6.fsaverage.%s.nii.gz' %'lh'))).get_data().squeeze()
    rh_dat = (nib.load(os.path.join(fast_dir, subject, fsd, run, 'fmcpr.sm6.fsaverage.%s.nii.gz' %'rh'))).get_data().squeeze()
    sc_dat = (nib.load(os.path.join(fast_dir, subject, fsd, run, 'fmcpr.sm6.%s.2mm.nii.gz' %('mni305')))).get_data()
    
    ## Drop first 4 volumes. 
    lh_dat = np.delete(lh_dat, np.arange(4), axis=1)
    rh_dat = np.delete(rh_dat, np.arange(4), axis=1)
    sc_dat = np.delete(sc_dat, np.arange(4), axis=3)

    n_times = sc_dat.shape[3]
    n_verts = lh_dat.shape[0]
    
    ## Remove any extra timepoints collected.
    if n_times > 176:
        print 'Removing last %d timepoints for %s' %(n_times-176,subject)
        lh_dat = np.delete(lh_dat, np.arange(176,n_times), axis=1)
        rh_dat = np.delete(rh_dat, np.arange(176,n_times), axis=1)
        sc_dat = np.delete(sc_dat, np.arange(176,n_times), axis=3)
        n_times = sc_dat.shape[3]
            
    elif n_times < 176: 
        print 'Mismatch in number of timepoints. %s, n_times: %d. Missing %d timepoints.' %(subject, n_times, 176-n_times)
        break
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Average by labels.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 
    mean_dat = np.zeros((n_regions,n_times))
    
    for idx, label in enumerate(lh_labels):
        mean_dat[idx] = lh_dat[label.vertices].mean(axis=0)
        
    for idx, label in enumerate(rh_labels): 
        mean_dat[idx+len(lh_labels)] = rh_dat[label.vertices].mean(axis=0)
            
    for idx,k in zip(np.arange(len(labels), n_regions),subcort_labels.keys()):
        x,y,z = subcort_labels[k]
        mean_dat[idx] = sc_dat[x,y,z].mean(axis=0)
        
    ## Save data averaged by labels. 
    np.savez_compressed(os.path.join(gt_dir, 'raw', '%s_raw' %subject), mean_dat=mean_dat)

print 'Done.'

0 CU0092CUMR1R1
1 CU0087CUMR1R1
2 CU0095CUMR1R1
3 CU0104CUMR1R1
4 CU0131CUMR1R1
5 CU0069CUMR1R1
6 CU0025CUMR1R1
7 CU0094CUMR1R1
8 CU0067CUMR1R1
9 CU0106CUMR1R1
10 CU0009CUMR1R1
11 CU0105CUMR1R1
12 CU0070CUMR1R1
13 CU0129CUMR1R1
14 CU0113CUMR1R1
15 CU0125CUMR1R1
16 CU0133CUMR1R1
Done.


### Filter and Nuisance Regress.

In [ ]:
from scipy.signal import detrend

## Specify directories. 
gt_dir = '/space/will/3/users/EMBARC/EMBARC-FAST/scripts/subtypes_gt'

subjects = [line.strip() for line in open("/space/will/3/users/EMBARC/EMBARC-FAST/scripts/subtypes_gt/gt_subjects", 'r')]

## Subjects with manual coregistration.
subjects = ['CU0092CUMR1R1', 'CU0087CUMR1R1', 'CU0095CUMR1R1', 'CU0104CUMR1R1', 'CU0131CUMR1R1', 'CU0069CUMR1R1', 
            'CU0025CUMR1R1', 'CU0094CUMR1R1', 'CU0067CUMR1R1', 'CU0106CUMR1R1', 'CU0009CUMR1R1', 'CU0105CUMR1R1', 
            'CU0070CUMR1R1', 'CU0129CUMR1R1', 'CU0113CUMR1R1', 'CU0125CUMR1R1', 'CU0133CUMR1R1']

for i, subject in enumerate(subjects[:1]):
    
    print i, subject
    
    ## Load data.
    mean_dat = np.load(os.path.join(gt_dir, 'raw', '%s_raw.npz' %subject))['mean_dat']
    
    ## Remove first 4 timepoints from data.
    mean_dat = np.delete(mean_dat, np.arange(4), axis=1)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Perform Nuissance Regression and Filtering.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    n_components = 5        # No. of components to use from PCA.

    ## Load global signal regressors. 
    gs = os.path.join(fast_dir, subject,fsd, run, 'global.waveform.dat')
    gs = read_table(gs, sep=' *', header=None, engine='python').as_matrix()
    
    ## Load wm and csf regressors. 
    wm = read_table(os.path.join(fast_dir, subject,fsd, run, 'wm.dat'), sep=' *', header=None, engine='python')
    vcsf = read_table(os.path.join(fast_dir, subject,fsd, run, 'vcsf.dat'), sep=' *', header=None, engine='python')
    
    ## Keep only top 5 components.
    n_components = 5 
    wm = wm[wm.columns[:n_components]].as_matrix()
    vcsf = vcsf[vcsf.columns[:n_components]].as_matrix()
    
    ## Remove extra timepoints. 
    if gs.shape[0] > 176: gs = np.delete(gs, np.arange(176,gs.shape[0]), axis=0)
    if wm.shape[0] > 176: wm = np.delete(wm, np.arange(176,wm.shape[0]), axis=0)
    if vcsf.shape[0] > 176: vcsf = np.delete(vcsf, np.arange(176,vcsf.shape[0]), axis=0)
    
    ## Load motion regressors.
    mc = os.path.join(fast_dir, subject, fsd, run, 'fmcpr.mcdat')
    mc = np.loadtxt(mc)[:,1:7]
    if mc.shape[0] > 176:
        mc = np.delete(mc, np.arange(176,mc.shape[0]), axis=0)

    ## Demean/detrend data.
    mc = detrend(mc, axis=0, type='constant')
    mc = detrend(mc, axis=0, type='linear') ## Keeps breaking

In [ ]:
    ## Construct basis sets.
    '''
    SAM'S NOTE(edited): Might want to fix the second step in the below three lines. 
    Probably want to insert a row of zeros into the last timepoints of the "rolled" 
    regressors, otherwise you are putting motion from the first frame into the last
    of the run
    '''
    mc = np.concatenate([mc, np.roll(mc, -1, 0)], axis=-1)
    mc = np.concatenate([mc, np.power(mc,2)], axis=-1)
    

In [ ]:
    ## Replace last timepoint with zeros 
    ## Otherwise you are putting motion from the first frame into the last of the run
    mc[-1] = np.zeros((mc[-1].shape))

    ## Apply PCA.  
    pca = PCA(n_components=24)
    mc = pca.fit_transform(mc)  

In [ ]:
    ## Concatenate nuisance regressors. 
    ns = np.hstack((mc,gs,wm,vcsf))
    
    ## Remove first 4 timepoints.
    ns = np.delete(ns, np.arange(4), axis=0) ## remove first 4 timepoints
    
    ## Filter parameters.
    tr = 3
    Fs = 1. / tr 
    Fp1 = 0.01
    Fp2 = 0.08

In [ ]:

    
    ## Filter data and regressors.
    mean_dat = band_pass_filter(mean_dat.astype(np.float64), Fs, Fp1, Fp2, filter_length='120s', method='iir')
    ns = band_pass_filter(ns.T, Fs, Fp1, Fp2, filter_length='120s', method='iir').T
    
    ## Regress. 
    betas,_,_,_ = np.linalg.lstsq(ns,mean_dat.T)
    regressed = mean_dat - np.dot(ns, betas).T
    
    ## Save regressed data.
    np.savez_compressed(os.path.join(gt_dir, 'regressed', '%s_regressed' %subject), regressed=regressed)
    
print 'Done.'

## Correlate.

In [ ]:
subjects = [line.strip() for line in open("/space/will/3/users/EMBARC/EMBARC-FAST/scripts/subtypes_gt/gt_subjects", 'r')]

for i, subject in enumerate(subjects):
    
    print i, subject
    
    ## Load regressed data. 
    regressed = np.load(os.path.join(gt_dir, 'regressed', '%s_regressed.npz' %subject))['regressed']
    
    ## Correlate. Remove negative correlations.
    corrmat = np.corrcoef(regressed)
    corrmat = np.where(corrmat<=0, np.nan, corrmat)
    
    # Fisher's r-to-z transform.
    zmat = np.arctanh(corrmat)
    zmat = np.where(zmat==np.inf,np.nan,zmat)

    ## Save correlation.
    np.savez(os.path.join(gt_dir, 'correlations', '%s_correlations' %subject), corrmat=corrmat)
    np.savez(os.path.join(gt_dir, 'correlations', '%s_zscores' %subject), zmat=zmat)

print 'Done.'

## Network Analysis.

### Construct Network.

In [ ]:
import os
import itertools
import numpy as np
import pylab as plt
import networkx as nx
import scipy.io as sio
%matplotlib inline

## Specify directories.
gt_dir = '/space/will/3/users/EMBARC/EMBARC-FAST/scripts/subtypes_gt'

## Load labels.
labels = [line.strip() for line in open("/space/will/3/users/EMBARC/EMBARC-FAST/scripts/subtypes_gt/gt_labels", 'r')]
n_labels = len(labels)

## Specify subjects.
subjects = [line.strip() for line in open("/space/will/3/users/EMBARC/EMBARC-FAST/scripts/subtypes_gt/gt_subjects", 'r')]
n_subs = len(subjects)

Gnets = []
for i,subject in enumerate(subjects): 
    zmat = np.load(os.path.join(gt_dir, 'correlations', '%s_zscores.npz' %subject))['zmat']
    
    ## Create graph.
    G = nx.Graph()
    G.add_nodes_from(labels)

    for i,x in enumerate(labels):
            for j,y in enumerate(labels):
                if not np.isnan(zmat[i,j]): 
                    G.add_edge(x,y)

    G.remove_nodes_from(G.nodes_with_selfloops())
    Gnets.append(G)
    
Gnets = np.array(Gnets)

### Compute global network metrics.

In [ ]:
## Network metrics. 
metrics = ['n_nodes', 'n_edges', 'avg_degree', 'avg_path', 'density', 'n_components']
metrics_dat = np.zeros((len(metrics), n_subs))

for i in range(n_subs):
    metrics_dat[0,i] = nx.number_of_nodes(Gnets[i])
    metrics_dat[1,i] = nx.number_of_edges(Gnets[i])
    metrics_dat[2,i] = sum(nx.degree(Gnets[i]).values())/Gnets[i].number_of_nodes()
    metrics_dat[3,i] = nx.average_shortest_path_length(Gnets[i])
    metrics_dat[4,i] = nx.density(Gnets[i])
    metrics_dat[5,i] = len([l for l in nx.connected_components(Gnets[i])])
    
## Average degree of nodes in G should be >= 2*np.log(G.number_of_nodes()) Bullmore 2006/Drakesmith 2015.
if not np.all(metrics_dat[2] > 2*np.log2(metrics_dat[0])): print 'Average degree of nodes not <= 2*log(#nodes)'    

### Plot distribution of global metrics.

In [ ]:
## Plotting
plt.figure(figsize=(24,18))
for r in range(len(metrics)):
    ax = plt.subplot2grid((2,3),(r/3,r%3),colspan=1,rowspan=1)
    ax.hist(metrics_dat[r], normed=True, bins=40)
    ax.set_title(metrics[r], fontsize=16)
plt.subplots_adjust(hspace=0.2)
plt.suptitle('Distribution of Graph Metrics (unthresholded)', fontsize=24)
plt.savefig('/space/will/3/users/EMBARC/EMBARC-FAST/scripts/subtypes_gt/plots/metrics_dist_unthresholded.png')

### Compute node-specific metrics.

In [ ]:
import pandas as pd

node_metrics = ['degree', 'clustering coefficient']

## Node-specific metrics.
degree_dist = pd.DataFrame(columns=labels,index=subjects)
clus_coeff = pd.DataFrame(columns=labels,index=subjects)
for i,sub in enumerate(subjects):
    degree_dist.loc[sub] = pd.Series(nx.degree(Gnets[i]))
    clus_coeff.loc[sub] = pd.Series(nx.clustering(Gnets[i]))
degree_dist = degree_dist.fillna(value=0)
clus_coeff = clus_coeff.fillna(value=0)

## Plotting Motion.
### Mask native timeseries.

In [ ]:
## Please see /space/will/3/users/EMBARC/EMBARC-FAST/scripts/subtypes_gt/wm_masks.csh to make wm masks.

In [ ]:
import os
import numpy as np
import nibabel as nib
from pandas import read_csv
from mne.filter import construct_iir_filter, filter_data
def demean(arr): return arr - arr.mean()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

decim = 250
# subjects = ['CU0016CUMR1R1']
subjects = [line.strip() for line in open("/space/will/3/users/EMBARC/EMBARC-FAST/scripts/subtypes_gt/gt_subjects", 'r')]

for subject in subjects:
    
    print subject

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare masks.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    mask_dir = '/space/will/3/users/EMBARC/EMBARC-FAST/%s/rest/001/masks' %subject
    mri_dir = '/space/will/3/users/EMBARC/EMBARC-FAST/%s/rest/001' %subject
    out_dir = '/space/will/3/users/EMBARC/EMBARC-FAST/%s/rest/001/motion' %subject

    brainmask = os.path.join(mask_dir,'brain.nii.gz')
    brainmask = np.where( nib.load(brainmask).get_data(), 1, 0 ) # Binarize the mask

    wm = os.path.join(mask_dir,'wm.mgz')
    wm = np.where( nib.load(wm).get_data(), 1, 0 ) # Binarize the mask

    gm = brainmask - wm

    ## Reduce to indices of interest.
    gm = np.vstack(np.where(gm))[:,::decim]
    wm = np.vstack(np.where(wm))[:,::decim]
    del brainmask

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and slice through EPI image.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load data.
    obj = nib.load(os.path.join(mri_dir, 'fmcpr.nii.gz'))
    _,_,_,n_acq = obj.shape

    ## Preallocoate space for timeseries.
    gmts = np.zeros((n_acq, gm.shape[-1]))
    wmts = np.zeros((n_acq, wm.shape[-1]))

    for n in range(n_acq):

        ## Slice image.
        acq = obj.dataobj[..., n]

        ## Store grey matter.
        gmts[n] += acq[gm[0],gm[1],gm[2]]

        ## Store white matter.
        wmts[n] += acq[wm[0],wm[1],wm[2]]

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Preprocessing data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ## Construct highpass filter.
    tr = 3
    sfreq = 1. / tr
    high_pass = 0.01
    iir_params = dict(order=2, ftype='butter', output='sos') # Following Power et al. (2014)
    iir_params = construct_iir_filter(iir_params, high_pass, None, sfreq, 'highpass', return_copy=False)  

    ## Filter data.
    gmts = filter_data(gmts.T, sfreq, high_pass, None, method='iir', iir_params=iir_params, verbose=False)
    wmts = filter_data(wmts.T, sfreq, high_pass, None, method='iir', iir_params=iir_params, verbose=False)

    ## De-mean.
    gmts = np.apply_along_axis(demean, 1, gmts)
    wmts = np.apply_along_axis(demean, 1, wmts)

    ## Re-organize (center outwards).
    gmts = gmts[ np.argsort( np.power( np.apply_along_axis(demean, 1, gm), 2 ).sum(axis=0) ) ]
    wmts = gmts[ np.argsort( np.power( np.apply_along_axis(demean, 1, wm), 2 ).sum(axis=0) ) ]

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Save data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    if not os.path.isdir(out_dir): os.makedirs(out_dir)
    f = os.path.join(out_dir, '%s_rest_qc_data' %subject)
    np.savez_compressed(f, gm=gmts, wm=wmts, iir_params=iir_params )
    del gmts, wmts, obj

print 'Done.'

### Prepare motion and masked fMRI data.

In [ ]:
import pylab as plt
from scipy.signal import detrend
from sklearn.decomposition import PCA
def demean(arr): return arr - arr.mean()
def rms(arr): return np.sqrt( np.mean( np.power(arr, 2) ) )
%matplotlib inline

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
threshold = 0.5
fast_dir = '/space/will/3/users/EMBARC/EMBARC-FAST'
fsd = 'rest'
run = '001'

subjects = [line.strip() for line in open("/space/will/3/users/EMBARC/EMBARC-FAST/scripts/subtypes_gt/gt_subjects", 'r')]

for subject in subjects[:1]:
    
    print subject, 
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Prepare motion data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Read motion data.
    mc = os.path.join(fast_dir, subject, fsd, run, 'fmcpr.mcdat')
    mc = np.loadtxt(mc)[:,1:7]

    ## Remove first four timepoints.
    mc = np.delete(mc, np.arange(4),axis=0)

    ## Invert angular displacement.
    copy = mc.copy()
    mc[:,:3] = np.deg2rad(mc[:,:3]) 
    mc[:,:3] *= 50

    ## Compute framewise displacement (See Power 2012, 2014).
    fd = np.insert( np.abs( np.diff(mc[:,3:], axis=0) ).sum(axis=1), 0, 0 )

    ## Demean/detrend data.
    mc = detrend(copy, axis=0, type='constant')
    mc = detrend(mc, axis=0, type='linear')

    ## Construct basis sets.
    mcreg24 = mc.copy()
    mcreg24 = np.concatenate([mcreg24, np.roll(mcreg24, -1, 0)], axis=-1)
    mcreg24 = np.concatenate([mcreg24, np.power(mcreg24,2)], axis=-1)

    ## Apply PCA.  
    pca = PCA(n_components=24)
    mcreg24 = pca.fit_transform(mcreg24)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Prepare fMRI data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load gray/white matter timeseries.
    npz = np.load(os.path.join(fast_dir, subject, fsd, run, 'motion/%s_rest_qc_data.npz' %subject))
    gm_pre = np.apply_along_axis(demean, 1, np.delete(npz['gm'],np.arange(4),axis=1))
    wm_pre = np.apply_along_axis(demean, 1, np.delete(npz['wm'],np.arange(4),axis=1))

    gm_beta, _, _, _ = np.linalg.lstsq(mcreg24, gm_pre.T)
    wm_beta, _, _, _ = np.linalg.lstsq(mcreg24, wm_pre.T)
    gm_post = gm_pre - np.dot(mcreg24, gm_beta).T
    wm_post = wm_pre - np.dot(mcreg24, wm_beta).T

    ## Compute DVARS.
    gm_DVARS = np.zeros((2,fd.shape[0]))
    wm_DVARS = np.zeros((2,fd.shape[0]))
    for n, arr in enumerate([gm_pre,gm_post]): gm_DVARS[n,1:] += np.apply_along_axis(rms, 0, np.diff(arr, axis=1))
    for n, arr in enumerate([wm_pre,wm_post]): wm_DVARS[n,1:] += np.apply_along_axis(rms, 0, np.diff(arr, axis=1))
            
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Plotting
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

In [21]:
    fig = plt.figure(figsize=(12,16))
    nrow = 19

    ## Plot framewise displacement.
    ax = plt.subplot2grid((nrow,1),(0,0),rowspan=1)
    ax.plot(fd, linewidth=1.5, color='k')
    ax.hlines(0.5, 0, fd.shape[0], linestyle='--', color='k', alpha=0.5)
    ax.set_xlim(0,fd.shape[0])
    ax.set_xticks([])
    ax.set_yticks([0.0,0.5,1.0])
    ax.set_yticklabels([0.0,0.5,1.0], rotation=90)
    ax.set_ylabel('FD (mm)', fontsize=12)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.set_title(subject, fontsize=24)

    ## Plot grey matter (pre-regression).
    ax = plt.subplot2grid((nrow,1),(1,0),rowspan=4)
    cbar = ax.imshow(gm_pre, aspect='auto', interpolation='none', origin='lower', cmap='bone', vmin=-50, vmax=50)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_ylabel('Grey Matter\nPre-Regression', fontsize=12)

    # Colobar setup.
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width, box.height])
    axColor = plt.axes([0.905, 0.238, 0.015, 0.660])
    cbar = plt.colorbar(cbar, cax = axColor, orientation="vertical")
    cbar.set_ticks([-50,0,50])
    axColor.set_ylabel('BOLD', fontsize=12, rotation=270)

    ## Plot grey matter (post-regression).
    ax = plt.subplot2grid((nrow,1),(5,0),rowspan=4)
    cbar = ax.imshow(gm_post, aspect='auto', interpolation='none', origin='lower', cmap='bone', vmin=-50, vmax=50)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_ylabel('Grey Matter\nPost-Regression', fontsize=12)

    ## Plot DVARS.
    ax = plt.subplot2grid((nrow,1),(9,0),rowspan=1)
    for arr, label in zip(gm_DVARS, ['Pre','Post']): ax.plot(arr, linewidth=1.5, alpha=0.8, label=label)
    ax.legend(loc=2, fontsize=8, frameon=False, borderpad=0.0,  handlelength=1.4, handletextpad=0.2)
    ax.set_xlim(0,fd.shape[0])
    ax.set_xlabel('Acquisitions', fontsize=12)
    ax.set_ylim(20,100)
    ax.set_yticks([20,60,100])
    ax.set_yticklabels([20,60,100], rotation=90)
    ax.set_ylabel('DVARS_gm', fontsize=12)
    ax.tick_params(axis='both', which='major', labelsize=12)

    ## Plot white matter (pre-regression).
    ax = plt.subplot2grid((nrow,1),(11,0),rowspan=2)
    cbar = ax.imshow(wm_pre, aspect='auto', interpolation='none', origin='lower', cmap='bone', vmin=-50, vmax=50)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_ylabel('White Matter\nPre-Regression', fontsize=12)

    ## Plot white matter (post-regression).
    ax = plt.subplot2grid((nrow,1),(13,0),rowspan=2)
    cbar = ax.imshow(wm_post, aspect='auto', interpolation='none', origin='lower', cmap='bone', vmin=-50, vmax=50)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_ylabel('White Matter\nPost-Regression', fontsize=12)

    ## Plot DVARS.
    ax = plt.subplot2grid((nrow,1),(15,0))
    for arr, label in zip(wm_DVARS, ['Pre','Post']): ax.plot(arr, linewidth=1.5, alpha=0.8, label=label)
    ax.legend(loc=2, fontsize=8, frameon=False, borderpad=0.0,  handlelength=1.4, handletextpad=0.2)
    ax.set_xlim(0,fd.shape[0])
    ax.set_xlabel('Acquisitions', fontsize=12)
    ax.set_ylim(20,100)
    ax.set_yticks([20,60,100])
    ax.set_yticklabels([20,60,100], rotation=90)
    ax.set_ylabel('DVARS_wm', fontsize=12)
    ax.tick_params(axis='both', which='major', labelsize=12)

    plt.subplots_adjust(left=0.05, right=0.90, top=0.95, bottom=0.05, hspace=0.05)
#     plt.show()
    plt.savefig(os.path.join(fast_dir,subject,fsd,run,'motion/%s_supp_motion.png' %subject), dpi=180)
    plt.close('all')
    
print 'Done.'

CU0002CUMR1R1 CU0014CUMR1R1 CU0016CUMR1R1 CU0024CUMR1R1 CU0025CUMR1R1 CU0029CUMR1R1 CU0033CUMR1R1 CU0034CUMR1R1 CU0036CUMR1R1 CU0039CUMR1R1 CU0040CUMR1R1 CU0046CUMR1R1 CU0047CUMR1R1 CU0051CUMR1R1 CU0052CUMR1R1 CU0056CUMR1R1 CU0057CUMR1R1 CU0059SBMR1R1 CU0060SBMR1R1 CU0061SBMR1R1 CU0067CUMR1R1 CU0069CUMR1R1 CU0070CUMR1R1 CU0071CUMR1R1 CU0072CUMR1R1 CU0074CUMR1R1 CU0077CUMR1R1 CU0078CUMR1R1 CU0081CUMR1R1 CU0082CUMR1R1 CU0083CUMR1R1 CU0085CUMR1R1 CU0087CUMR1R1 CU0089CUMR1R1 CU0090CUMR1R1 CU0092CUMR1R1 CU0093CUMR1R1 CU0094CUMR1R1 CU0095CUMR1R1 CU0097CUMR1R1 CU0100CUMR1R1 CU0102CUMR1R1 CU0103CUMR1R1 CU0104CUMR1R1 CU0105CUMR1R1 CU0106CUMR1R1 CU0108CUMR1R1 CU0110CUMR1R1 CU0113CUMR1R1 CU0115CUMR1R1 CU0117CUMR1R1 CU0119CUMR1R1 CU0120CUMR1R1 CU0125CUMR1R1 CU0126CUMR1R1 CU0128CUMR1R1 CU0129CUMR1R1 CU0130CUMR1R1 CU0131CUMR1R1 CU0133CUMR1R1 MG0018MGMR1R1 MG0021MGMR1R1 MG0032MGMR1R1 MG0039MGMR1R1 MG0060MGMR1R1 MG0064MGMR1R1 MG0069MGMR1R1 MG0076MGMR1R1 MG0101MGMR1R1 MG0106MGMR1R1 MG0112MGMR1R1 MG0116